In [ ]:
import glob
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os
import csv
import torch.optim as optim
from PIL import Image
from torchvision import transforms
import csv
import torchvision.transforms as T

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip train_small.zip

Archive:  train_small.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of train_small.zip or
        train_small.zip.zip, and cannot find train_small.zip.ZIP, period.


In [ ]:
!unzip test.zip

Archive:  test.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of test.zip or
        test.zip.zip, and cannot find test.zip.ZIP, period.


In [ ]:
!unzip gdrive/My\ Drive/train_small.zip

unzip:  cannot find or open gdrive/My Drive/train_small.zip, gdrive/My Drive/train_small.zip.zip or gdrive/My Drive/train_small.zip.ZIP.


In [ ]:
label = []
names = []
with open('category.csv', 'r') as file:
  reader = csv.reader(file)
  for row in reader:
    names.append(row[1])
    label.append(row[0])


In [ ]:
del label[0]
del names[0]


In [ ]:
print(label)
print(names)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']
['Art Carney', 'Akemi Darenogare', 'Billy Bob Thornton', 'Bill Daley', 'Amanda Bynes', 'Anna Friel', 'Alexander Skarsgard', 'Anna Faris', 'Bill Pullman', 'Anjelica Huston', 'Angela Bassett', 'Alx James', 'Benjamin Bratt', 'Andy Garcia', 'Bernie Mac', 'Andreea Diaconu', 'Anna Kendrick', 'Anna Sui', 'Barry Pepper', 'Alexis Thorpe', 'Ashley Judd', 'Amy Ryan', 'Analeigh Tipton', 'Anne Hathaway', 'Amanda Peet',

In [ ]:
class CustomDataset(Dataset):
    def __init__(self , label , names):
        self.imgs_path = "train.csv"
        self.folder = 'train/'
        self.data = []
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224,224)),
            #transforms.RandomIoUCrop(),
            transforms.RandomHorizontalFlip(),
            ])
        with open(self.imgs_path , 'r') as file:
          reader = csv.reader(file)
          for row in reader:
            file_name = row[1]
            dir = os.path.join(self.folder + file_name)
            name = row[2]
            if(name != 'Category'):
              name_index = names.index(name)
              label_category = label[name_index]
              #print(label_category)
              self.data.append([dir , label_category])


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      image_path , index = self.data[idx]
      #print(index)
      image = Image.open(image_path)
      #jitter = T.ColorJitter(brightness=.5, hue=.3)
      #image = jitter(image)
      try:
        image = image.convert('RGBA')
      except:
        print("didnt convert")
      try:
        im_tensor = self.transform(image)
      except:
        print("didnt convert")
      #print(im_tensor)
      return im_tensor , int(index)




In [ ]:
dataset = CustomDataset(label,names)
train_loader = DataLoader(dataset, batch_size=25, shuffle=True)

In [ ]:
len(dataset)

69540

Make some model

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = stride, padding = 1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU())
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1),
                        nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 100):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(4, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU())
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        self.avgpool = nn.AvgPool2d(7, stride=1)

        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes),
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)


    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
num_classes = 100
num_epochs = 30
batch_size = 16
learning_rate = 0.005

model = ResNet(ResidualBlock, [3, 4, 6, 3])
model = model.cuda()
labeling_loss_tally = []
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.001, momentum = 0.9)

import gc
total_step = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.cuda()
        labels = labels.cuda()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, loss.item()))

    # Validation
 #   with torch.no_grad():
  #      correct = 0
   #     total = 0
    #    for images, labels in train_loader:
     #       images = images.cuda()
      #      labels = labels.cuda()
       #     outputs = model(images)
        #    _, predicted = torch.max(outputs.data, 1)
        #    total += labels.size(0)
         #   correct += (predicted == labels).sum().item()
         #   del images, labels, outputs

        #print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))




/usr/local/lib/python3.9/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch [1/30], Loss: 3.8029
Epoch [2/30], Loss: 3.7179
Epoch [3/30], Loss: 3.6088
Epoch [4/30], Loss: 3.1501
Epoch [5/30], Loss: 2.5710
Epoch [6/30], Loss: 2.2889
Epoch [7/30], Loss: 2.0894
Epoch [8/30], Loss: 2.3227
Epoch [9/30], Loss: 1.2282
Epoch [10/30], Loss: 1.7838
Epoch [11/30], Loss: 2.2296
Epoch [12/30], Loss: 2.0916
Epoch [13/30], Loss: 3.0915
Epoch [14/30], Loss: 1.4131
Epoch [15/30], Loss: 1.5532
Epoch [16/30], Loss: 2.3363
Epoch [17/30], Loss: 1.6843
Epoch [18/30], Loss: 1.5374
Epoch [19/30], Loss: 1.6878
Epoch [20/30], Loss: 0.9663
Epoch [21/30], Loss: 1.4687
Epoch [22/30], Loss: 1.5531
Epoch [23/30], Loss: 1.2245
Epoch [24/30], Loss: 0.7604
Epoch [25/30], Loss: 1.4832
Epoch [26/30], Loss: 1.9810
Epoch [27/30], Loss: 1.3528
Epoch [28/30], Loss: 0.4740
Epoch [29/30], Loss: 0.8864
Epoch [30/30], Loss: 1.3947


Validation

In [ ]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for data in train_loader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy on test set: %.2f %%' % (100 * correct / total))

In [ ]:
!unzip test.zip

In [ ]:
class CustomDataset(Dataset):
    def __init__(self):
        #self.imgs_path = "train.csv"
        self.root = 'test'
        self.data = []
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize((224,224)),
            #transforms.RandomIoUCrop(),
            transforms.RandomHorizontalFlip(),
            ])

        self.filenames = sorted([os.path.join(self.root, f) for f in os.listdir(self.root) if f.endswith('.jpg')])
        for image_path in glob.glob(self.root + "/*.jpg"):
          self.data.append(image_path)


    def __len__(self):
        return len(self.data)

    def __getitem__(self , idx):
      image_path = self.filenames[idx]
      #print(image_path)
      image = Image.open(image_path)
      image = image.convert('RGBA')
      im_tnsr = self.transform(image)
      return im_tnsr

In [ ]:
dataset = CustomDataset()
test_loader = DataLoader(dataset, batch_size=25)
len(dataset)

4977

In [ ]:
#correct = 0
#total = 0
import matplotlib.pyplot as plt
model.eval()
preds = []
with torch.no_grad():
    for data in test_loader:
        images = data
        images = images.cuda()
        #image_np = images[0].detach().cpu().numpy()
        #img_np = np.transpose(image_np, (1, 2, 0)).clip(0,1)
        #img_np = np.transpose(image_np, (1, 2, 0)).clip(0,1)
        #plt.imshow(img_np)
        #plt.show()
        #labels = labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)

        preds.append(predicted)



tensor([18, 21, 42, 89, 23, 12,  5, 54, 41, 81,  7, 59, 34, 90, 48, 91, 32, 49,
         3, 31, 31, 67,  1, 94, 89], device='cuda:0')
tensor([20, 40, 61, 86, 43, 93, 51, 36, 93, 88, 68, 19, 35, 40, 55, 11, 23, 25,
         9, 16, 40, 87,  4, 34, 30], device='cuda:0')
tensor([ 3, 55, 59, 89, 89, 89, 99, 55, 45, 71, 40, 78, 37, 31, 67, 57, 27, 11,
        19, 42, 31, 64, 40, 12, 50], device='cuda:0')
tensor([64, 61, 99, 63, 51, 89, 97, 49, 26, 42, 25, 62, 56, 32, 26, 27, 20, 56,
        42, 49, 93, 20, 20, 78, 40], device='cuda:0')
tensor([35, 26,  1, 90, 67, 41, 36, 69, 81, 48, 19, 51, 48, 93, 89,  2, 88,  2,
        45,  8, 62, 93, 27,  5, 81], device='cuda:0')
tensor([59,  1, 61, 57, 57, 57, 81, 53, 62, 14,  5, 96, 53, 51, 24, 67, 63,  3,
        49,  4, 84, 72, 36,  6,  3], device='cuda:0')
tensor([64, 15, 23, 98, 79, 87, 18, 37, 14, 16, 19, 81, 22,  2, 65, 65, 56, 70,
        94, 58, 33, 31, 50, 33, 18], device='cuda:0')
tensor([67, 34, 64, 89, 14, 68, 18, 20, 14, 34,  1, 26, 26, 30

In [ ]:
pre = []

for batch_images in preds:
    pre.extend(batch_images)

len(pre)

4977

In [ ]:
print(names[19])

Alexis Thorpe


In [ ]:
for i in pre:
  print(i.item())

In [ ]:
names_pred = []
for i in pre:
  print(i.item())
  n = names[i.item()]
  print(n)
  names_pred.append(n)



In [ ]:
file_path = 'pred_submit.csv'

# Open the CSV file for writing
with open(file_path, 'w', newline='') as csvfile:
    # Create a writer object
    writer = csv.writer(csvfile)

    # Write the header row
    writer.writerow(['Id', 'Category'])

    # Write the data rows
    for i, name in enumerate(names_pred, start=0):
        writer.writerow([i, name])

Bigger dataset

In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c 'purdue-face-recognition-challenge-2023'

100% 562k/562k [00:00<00:00, 979kB/s]
100% 562k/562k [00:00<00:00, 978kB/s]


In [ ]:
!unzip purdue-face-recognition-challenge-2023.zip

Archive:  purdue-face-recognition-challenge-2023.zip
  inflating: category.csv            
  inflating: sample_submission.csv   
  inflating: train.csv               
  inflating: train_small.csv         
